### What we have tried
* we have tried letting the gan have no penalties for certian actions and only for not being close to the discriminator:
this results in all of the points grouping together most likely because the discriminator say there was features and that the graphs had edges and felt that it was a good results

* we tried adding penilites if the node features (x,y) where too close together, this resulted in the data spreading out, but was most likely a band-aid and ulimatley was not teaching the dsciriminator much as it just outputed random patterns of spread data

### Ongoing
* creating a boid class for every boid that contain ['Boid_id','x','y','dx','dy'] in order to be able to track how the poids postions are evolving over time

### What should we try in the future?


## What to do
### Graphs
* what are graphs?
* why are they useful
* what is our use case for them

### Model
* explain what GAN is and why we want to use it
* explain dataset gen(dont do until model is finished)
* explain decoder and encoder
* explain  discirmintor 
* explain traiing loop and ways loss was implemented into gan
* explain results(cannot do righ now.)

## What to do
### Graphs
* what are graphs?
* why are they useful
* what is our use case for them


- graphs are a  form of representation for data that posses qualities that make it useful for problems that require connections.
- Examples (socail systems, molecules, )
- they consists of nodes and edges which can have features within them. Nodes are your classes (in our case we have a homogenous graph with all of the same class boids), and edges are the connections between our nodes. Eges are a binary object (either there is one or there isn't between nodes) but within an edge you can inlcude edges features, similiar to nodes, features contain useful data about the node or edge that may be useful in different types of predictions. An example being edges reprepsent bonds between moleucles, and one of the edge features could be the strength of the bond.
- For our problem we want to use graphs to represent the connections flocking birds have. For this project we are assuming that the birds interactions are based off of its nearest neighbors since they are the only things it can see in its vicinity. our nodes for each boid consist of features like (x,y,dx,dy,d^2x, d^2y) and our edges are made based off of a threshold distance and the features within the edges are the distance between the boids and the angel between them (** this is subject to change as we improve our model).


### Model
* explain what GAN is and why we want to use it
* explain dataset gen(dont do until model is finished)
* explain decoder and encoder
* explain  discirmintor 
* explain traiing loop and ways loss was implemented into gan
* explain results(cannot do righ now.)


* General adversial Newtorks (Gans) are a form of unsupervised learning that at a basic level consists of a generator and discrimnator. The job of the generator is to create data that fools the discrimintor, and the discriminotr jobs is to try and accuralty guess if the data being inputed into it is fake or real. Both real and fake data is inputed in the disrimintor. 
* dataset generation
* The reaon we want to use a gan is it allows our model (the geneator) to learn things abour our data that we may not have thought about or known to look for, that will potential make our model capture these hidden interactions bewteen boids as they flock. Furthmore, GAN or useful for generative models ( making new data). Our goal is to leverage the abilites of the GAN to predicct the next postions of boids in a sequances (t+1). We input a seqaunce of a certain length, and are attempting to predcit what the next prediction will be. (** make a diagram showing what you mean with a sliding window and also clarify that the generator is not predcitong all the data at once, but instead using a sequence and predcint the next time step, then we take every one of these predicted timesteps to create a new simulation. )



In [ ]:
all_edge_indices_test, all_node_features_test = process_dfs(testing_20, testing_edges_20)
all_edge_indices, all_node_features = process_dfs(sim_df_final, sim_edges_df_final)

combinations = list(itertools.combinations(range(100), 2))
all_possible_edge_indices = np.array(combinations).T
all_possible_edge_indices.shape

ll_edge_attr_test = compute_edge_attributes(testing_20)
all_edge_attr = compute_edge_attributes(sim_df_final)


final_min_node_feat_test, final_max_node_feat_test = compute_feature_min_max(all_node_features_test)
final_min_edge_attr_test, final_max_edge_attr_test= compute_feature_min_max(all_edge_attr_test)
final_min_node_feat, final_max_node_feat = compute_feature_min_max(all_node_features)
final_min_edge_attr, final_max_edge_attr = compute_feature_min_max(all_edge_attr)

all_node_features_normalized = minmax_scale(all_node_features, final_min_node_feat, final_max_node_feat)
all_edge_attr_normalized = minmax_scale(all_edge_attr, final_min_edge_attr, final_max_edge_attr)
all_node_features_normalized_test = minmax_scale(all_node_features_test, final_min_node_feat_test, final_max_node_feat_test)
all_edge_attr_normalized_test= minmax_scale(all_edge_attr_test, final_min_edge_attr_test, final_max_edge_attr_test)

node_feature_sequences, node_feature_targets = split_dataset(all_node_features_normalized, window=5, horizon=1)
edge_weights_sequences, edge_weights_targets = split_dataset(all_edge_attr_normalized, window=5, horizon=1)
edge_indices_sequence = [all_possible_edge_indices for _ in range(len(node_feature_sequences))]
node_feature_sequence_test, node_feature_targets_test = split_dataset(all_node_features_normalized_test, window=5, horizon=1)
edge_weights_sequences_test, edge_weights_targets_test = split_dataset(all_edge_attr_normalized_test, window=5, horizon=1)
edge_indices_sequence_test = [all_possible_edge_indices for _ in range(len(node_feature_sequence_test))]

train_data = CustomStaticGraphTemporalSignal(edge_index=edge_indices_sequence, 
                                       edge_weight=edge_weights_sequences, 
                                       features=node_feature_sequences, 
                                       targets=node_feature_targets
                                       )

test_data = CustomStaticGraphTemporalSignal(edge_index=edge_indices_sequence_test, 
                                       edge_weight=edge_weights_sequences_test, 
                                       features=node_feature_sequence_test, 
                                       targets=node_feature_targets_test
                                       )
